In [78]:
import pandas as pd
import numpy as np

In [79]:
pdfminer = pd.read_pickle('pdfminer_ables.dat')
pytess = pd.read_pickle('pytesseract_results.dat')
pdfminer['extracted_text'] = pdfminer['de_headed']
pytess['extracted_text'] = pytess['reparse']
combined = pytess.combine_first(pdfminer)
combined = combined[['filename','extracted_text']]

In [80]:
def spacing(docu):
    docu = docu.replace('  ',' ')
    docu = docu.replace('   ',' ')
    return docu

In [81]:
def lower(docu):
    docu = docu.lower()
    return docu

In [82]:
def read_address(docu, pin0= 'addresses', pin1 = 'start', pin2 = 'end'):
    start = docu.find(pin0)
    borrower = docu[start:].find(pin1)
    colon = docu[start+borrower:].find(':')
    bank = docu[start+borrower+colon:].find(pin2)
    add = docu[start+borrower+colon+1:start+borrower+colon+bank]
    if 'in witness' in add:
        end2 = docu[start+borrower+colon+1:].find('in witness')
        add = add[:end2]
    return add   

In [83]:
address1 = []
for i in range(len(combined)):
    docu = combined.iloc[i,1]
    docu = spacing(docu)
    docu = lower(docu)
    address = read_address(docu, pin0 = 'addresses', pin1 = 'for the borrower', pin2 = 'for the bank')
    if len(address) < 8:
        address = read_address(docu, pin0 = 'addresses', pin1 = 'for the borrower', pin2 = 'international bank')
    address1.append(address)

In [84]:
address2 = []
for i in range(len(combined)):
    docu = combined.iloc[i,1]
    docu = spacing(docu)
    docu = lower(docu)
    address = read_address(docu, pin0 = 'addresses', pin1 = 'address is', pin2 = 'the bank')
    address2.append(address)


In [85]:
address3 = []
for i in range(len(combined)):
    docu = combined.iloc[i,1]
    docu = spacing(docu)
    docu = lower(docu)
    address = read_address(docu, pin0 = 'addresses', pin1 = 'the borrower', pin2 = 'the world bank')
    address3.append(address)

In [86]:
for i in range(len(address1)):
    if len(address1[i]) < 8 & len(address2[i]) > 8:
        address1[i] = address2[i]
    elif len(address1[i]) > 500:
        address1[i] = address2[i]

In [87]:
for i in range(len(address1)):
    if len(address1[i]) < 8 & len(address3[i]) > 8:
        address1[i] = address3[i]

In [88]:
data = pd.read_pickle('../Extracted_Information.dat')
data['Address'] = pd.Series(address1)

In [94]:
data.to_pickle('Extracted_Information.dat')

In [97]:
a = 0
for i in range(len(address1)):
    if len(address1[i]) < 8:
        a = a+1

In [98]:
a

113

In [100]:
b = data['Address'].iloc[0]

In [101]:
len(b)

133